In [2]:
import pandas as pd

___ПРИЗНАКИ: КАТЕГОРИАЛЬНЫЕ И ЧИСЛОВЫЕ___

Под ___числовыми___ признаками обычно подразумевают признаки, которые отражают количественную меру и могут принимать значения из неограниченного диапазона.

Числовые признаки могут быть:
- _дискретными_ (например, количество комнат, пациентов, дней);  
- _непрерывными_ (например, масса, цена, площадь).  

Дискретные признаки чаще всего представлены целыми числами, а непрерывные — целыми числами и числами с плавающей точкой.

Под ___категориальными___ признаками обычно подразумевают столбцы в таблице, которые обозначают принадлежность объекта к какому-то классу/категории.

Категориальные признаки могут быть:
- _номинальными_ (например, пол, национальность, район);  
- _порядковыми_ (например, уровень образования, уровень комфорта, стадия заболевания).  

Такие признаки имеют ограниченный набор значений. Они чаще всего представлены в виде текстового описания и кодируются в Pandas типом данных _object_.

In [3]:
melb_data = pd.read_csv('melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

In [4]:
unique_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(), melb_df[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(
    unique_list,
    columns = ['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique', ignore_index=True)
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Rooms,9,int64
4,Bathroom,9,int64
5,Car,11,int64
6,Bedroom,12,int64
7,CouncilArea,33,object
8,Date,58,object
9,YearBuilt,144,int64


Самый простой способ преобразования столбцов к типу данных _category_ — это использование уже знакомого нам метода _astype()_, в параметры которого достаточно передать строку _'category'_.

In [ ]:
display(melb_df.info())
# 2.4+ Mb

In [8]:
cols_to_exclude = ['Date', 'Rooms', 'Bedrooms', 'Bathroom', 'Car']
max_unique_count = 140
for col in melb_df:
     if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude:
        melb_df[col] = melb_df[col].astype('category')
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  category
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  int64   
 17  CouncilArea    12211 non-null  

None

У типа данных _category_ есть свой специальный аксесcор _cat_, который позволяет получать информацию о своих значениях и преобразовывать их. Например, с помощью атрибута этого аксессора _categories_ мы можем получить список уникальных категорий в столбце Regionname

In [19]:
print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


In [18]:
display(melb_df['Regionname'].cat.codes)
# узнать как столбец кодируется в виде чисел можно с помощью атрибута codes

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

С помощью метода аксессора _rename_categories()_ можно легко переименовать текущие значения категорий. Для этого в данный метод нужно передать _словарь, ключи которого — старые имена категорий, а значения — новые_.

Рассмотрим на примере: переименуем категории признака типа постройки Type — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).

In [20]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u':'unit',
    't':'townhouse',
    'h':'house'
})
display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

___ПОДВОДНЫЕ КАМНИ___  
А теперь представим ситуацию, что появилась новая партия домов и теперь мы продаём и квартиры (flat). Создадим объект Series new_houses_types, в котором будем хранить типы зданий новой партии домов. Преобразуем тип new_houses_types в такой же тип, как и у столбца Type в таблице melb_data

In [25]:
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
# появился тип NaN вместо новых значений
display(new_houses_types)

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

Тип данных _category_ хранит только категории, которые были объявлены при его инициализации. При встрече с новой, неизвестной ранее категорией, этот тип превратит её в пустое значение, так как он просто не знает о существовании этой категории.  
Решить эту проблему на самом деле не сложно. Можно добавить категорию _flat_ в столбец _Type_ с помощью метода акссесора __cat__ _add_categories()_, в который достаточно просто передать имя новой категории

In [ ]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

__Рекомендации по использования category__  
__1.__ Необязательно каждый раз преобразовывать категориальные данные в тип данных category. Зачастую это делается исключительно для оптимизации работы с большими данными.

__2.__ Если набор данных занимает значительный процент используемой оперативной памяти, рассмотрите возможность использования типа category.

__3.__ Если у вас очень серьёзные проблемы с производительностью, обратите внимание на использование типа category.

__4.__ Если вы решили использовать тип category, будьте осторожны при добавлении новой информации в вашу таблицу. Убедитесь, что вы собрали всю необходимую информацию, произведите предобработку данных и только после этого используйте преобразование типов.

In [29]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  category
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  int64   
 17  CouncilArea    12211 non-null  

5.2
С помощью метода info() узнайте, сколько памяти занимает таблица melb_df.  
Преобразуйте признак Suburb следующим образом: оставьте в столбце только 119 наиболее популярных пригородов, остальные замените на 'other'.  
Приведите данные в столбце Suburb к категориальному типу.

In [39]:
# 5.2
popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  category
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  category
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  int64   
 17  CouncilArea    12211 non-null  

None